Завдання 1

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357275 sha256=91009752f83f5ed1d60bb42da2b7e1556d4a30cca45e5eb50e549ae8e074df58
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise.accuracy import mae

# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
data = Dataset.load_builtin('ml-100k')

# 2. Виведіть перші 5 рядків завантаженого датасету.
raw_ratings = data.raw_ratings[:5]
print("Перші 5 рядків датасету:")
for line in raw_ratings:
    print(line)

# 3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету.
algo1 = SVD()
algo2 = KNNBasic()

# 4. Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print("Найкращі параметри для SVD:")
print(gs.best_params['mae'])

algo1 = gs.best_estimator['mae']

param_grid = {'k': [20, 40], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [False]}}
gs = GridSearchCV(KNNBasic, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print("Найкращі параметри для KNNBasic:")
print(gs.best_params['mae'])

algo2 = gs.best_estimator['mae']

# 5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки (MAE).
results1 = cross_validate(algo1, data, measures=['mae'], cv=5, verbose=True)
results2 = cross_validate(algo2, data, measures=['mae'], cv=5, verbose=True)

mean_mae_algo1 = results1['test_mae'].mean()
mean_mae_algo2 = results2['test_mae'].mean()

best_algo = algo1 if mean_mae_algo1 < mean_mae_algo2 else algo2
print(f"Найкращий алгоритм: {'SVD' if best_algo == algo1 else 'KNNBasic'}")

# 6. Виведи рекомендації (10 фільмів) для конкретного користувача.
trainset = data.build_full_trainset()
best_algo.fit(trainset)

user_id = str(196)  # Заміни на ID користувача, для якого потрібні рекомендації
items = trainset.all_items()
items = [trainset.to_raw_iid(i) for i in items]
predictions = [best_algo.predict(user_id, iid) for iid in items]
predictions.sort(key=lambda x: x.est, reverse=True)

print("Рекомендації для користувача:")
for pred in predictions[:10]:
    print(pred.iid, pred.est)


Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k
Перші 5 рядків датасету:
('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')
Найкращі параметри для SVD:
{'n_epochs': 10, 'lr_all': 0.005}
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity mat

Завдання 2

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise.accuracy import mae

# 1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
data = Dataset.load_builtin('ml-100k')

# Розділіть дані на тренувальний та тестовий набори
trainset, testset = train_test_split(data, test_size=0.25)

# Створіть алгоритм SVD
algo = SVD()

# Навчіть алгоритм на тренувальному наборі
algo.fit(trainset)

# Оцініть алгоритм на тестовому наборі
predictions = algo.test(testset)
print("MAE:", mae(predictions))

# Використайте крос-валідацію для оцінки алгоритму
cross_validate(algo, data, measures=['MAE'], cv=5, verbose=True)

# 2. Отримайте рекомендацію для певного користувача.
trainset = data.build_full_trainset()
algo.fit(trainset)

user_id = str(196)  # Заміни на ID користувача, для якого потрібні рекомендації
items = trainset.all_items()
items = [trainset.to_raw_iid(i) for i in items]
predictions = [algo.predict(user_id, iid) for iid in items]
predictions.sort(key=lambda x: x.est, reverse=True)

print("Рекомендації для користувача:")
for pred in predictions[:10]:
    print(pred.iid, pred.est)


MAE:  0.7423
MAE: 0.742339085349967
Evaluating MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.7410  0.7355  0.7356  0.7390  0.7346  0.7371  0.0024  
Fit time          1.37    1.37    1.39    1.49    1.96    1.52    0.23    
Test time         0.27    0.12    0.13    0.20    0.18    0.18    0.05    
Рекомендації для користувача:
318 4.624020065469233
134 4.577526363701276
64 4.54933003832522
169 4.542567131960868
408 4.5348173906993585
474 4.490454864801153
178 4.464608803561293
479 4.460454887252557
663 4.446295720072948
515 4.412340139059769


**Висновки**

1. **Завантаження та підготовка даних**:
   - Використано датасет `ml-100k` з бібліотеки `surprise`.
   - Дані розділено на тренувальний та тестовий набори для навчання та оцінки моделі.

2. **Реалізація алгоритму**:
   - Використано алгоритм SVD для побудови рекомендаційної системи.
   - Алгоритм навчено на тренувальному наборі даних.

3. **Оцінка моделі**:
   - Оцінка моделі проведена за допомогою метрики середньої абсолютної помилки (MAE).
   - Використано крос-валідацію для додаткової оцінки алгоритму, що дозволило отримати більш надійні результати.

4. **Отримання рекомендацій**:
   - Після навчання моделі на повному наборі даних, отримано рекомендації для конкретного користувача.
   - Виведено топ-10 рекомендованих фільмів для користувача з ID 196.

Цей підхід дозволяє створити ефективну рекомендаційну систему, яка може надавати персоналізовані рекомендації на основі історичних даних користувачів.